In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time
import random

In [ ]:
actorsDataFrame = pd.read_csv('unique_actors.csv')
actors = actorsDataFrame['actors'].tolist()
#JON: 0:4594
#JAMES: 4595:9190
#ALEX: 9191:13785
#ROB: 13786:18372
actors = actors[0: 4594]


In [ ]:
chrome_path = "./chromedriver"
options = webdriver.ChromeOptions()
options.add_argument('--load-extension=./adblock4Sellenium/3.34.0_0')
options.add_experimental_option( "prefs",{'profile.managed_default_content_settings.javascript': 2})
# options.add_argument("--user-agent=" + 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')
driver = webdriver.Chrome(chrome_path, options=options)
                            

In [ ]:
actorDataFrame = []

In [ ]:
#Finds number of movies from soup
def findNumMovies(soup):
    numCreds = soup.find("div", {"id": "filmo-head-actor"})
    if numCreds is None:
        numCreds = soup.find("div", {"id": "filmo-head-actress"})
    rawNumOfMovies = numCreds.text
    return rawNumOfMovies

In [ ]:
#under: call function to web scrape the page
def scrape(actorName, htmlURL):
    #try:
        html_doc = requests.get(htmlURL)
        soup = BeautifulSoup(html_doc.text, 'html.parser')
        awards = soup.findAll("span", {"class": "awards-blurb"})
        rawNumOfMovies = findNumMovies(soup)    
    
        rawNumOfMovies = re.findall('\d+', rawNumOfMovies)
        numMovies = int(rawNumOfMovies[0])
        if len(awards) == 0:
            numWins = 0
            numNoms = 0 
        elif len(awards) == 1 :
            #when no oscars and maybe having just a win / nomination on its own
            rawWinsAndNoms = awards[0].text
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            if ("wins" in rawWinsAndNoms or "win" in rawWinsAndNoms) and ("nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms):
                numWins = int(winsAndNoms[0])
                #a
                numNoms = int(winsAndNoms[1])
            elif "wins" in rawWinsAndNoms or "win" in rawWinsAndNoms:
                numWins = int(winsAndNoms[0])
                numNoms = numWins
            elif "nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms:
                numNoms = int(winsAndNoms[0])
                numWins = 0
            #When they have an oscar and other nominations
        else:
            #TODO add the 2 oscars etc.. to the total number    
                
            rawWinsAndNoms = awards[1].text
    
            # winsAndNoms = re.findall('\\s\\d+\\s', rawWinsAndNoms)
            winsAndNoms = re.findall('\d+', rawWinsAndNoms)
            #If they have both wins and nominations available
            if ("wins" in rawWinsAndNoms or "win" in rawWinsAndNoms) and ("nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms):
                numWins = int(winsAndNoms[0])
                numNoms = int(winsAndNoms[1])
            #if they have only one of the two available
            else:
                if "wins" in rawWinsAndNoms or "win" in rawWinsAndNoms:
                    numWins = int(winsAndNoms[0])
                    numNoms = numWins
                elif "nominations" in rawWinsAndNoms or "nomination" in rawWinsAndNoms:
                    numNoms = int(winsAndNoms[0])
                    numWins = 0
            oscarWins = awards[0].text
            oscarWins = re.findall('\d+', oscarWins)
            oscarWins = int(oscarWins[0])
            numWins += oscarWins
            numNoms += oscarWins
                        
    
    
            # List of the Rating Values (in order; number of wins, num of nominations, and num of movies):
            #actorValues = [numWins, numNoms, numMovies]
        print(actorName, ": Movies: ", numMovies, " Noms: ", numNoms, " Wins: ", numWins)
    
    
        actorDataFrame.append({'actorName': actorName, 'actorMovieCount': numMovies,
                            'actorAwardNominations': numNoms, 'actorAwardWins': numWins})
    #except:
        #For all errors that show up after we have all edge cases we should set movies to 1 and others to 0
            #print('***Error with scraping for: ' + name + '. Skipping to next actor.***')
        

In [ ]:
#TODO: fix when ads come up.
count = len(actors)
    
driver.get('https://www.imdb.com/?ref_=nv_home')
for name in actors:
    waitTime = random.uniform(0.5,1.5)
    time.sleep(waitTime)
    if count % 2 == 0:
        print("number of actors remaining: %i" % count)
    #going to main page of IMDB
    #Find search bar, clear and search for actor
    try:
        searchBar = driver.find_element_by_id('navbar-query')
        searchBar.clear()
        searchBar.send_keys(name)
        searchBar.send_keys(Keys.RETURN)
        try:
            #Taking the top actor from the list of names that show up from the search
            personWanted = driver.find_element_by_class_name('findResult')
            personURL = personWanted.find_element_by_tag_name('a')
            personURL.click()
            driver.current_url
            scrape(name, driver.current_url)
        #except instance when a search of an actor has a non-actor (ex: show) as the top choice
        except:
            #Reason for doing a whole repeat is because of HTML format where cannot determine if link to click on is actor or not
            #Find search bar, clear and search for actor
            searchBar = driver.find_element_by_id('navbar-query')
            searchBar.clear()
            searchBar.send_keys(name)
            searchBar.send_keys(Keys.RETURN)
            assert "No results found." not in driver.page_source
            #Taking the top actor from the list of names that show up from the search
            moreName = driver.find_element_by_partial_link_text('More name matches')
            moreName.click()
            personWanted = driver.find_element_by_class_name('findResult')
            personURL = personWanted.find_element_by_tag_name('a')
            personURL.click()
            driver.current_url
            scrape(name, driver.current_url)
            count -= 1
    except:
        print("Actor: ", name," not searchable")
   
actorDataFrame = pd.DataFrame(actorDataFrame, columns = ['actorName', 'actorMovieCount', 'actorAwardNominations', 'actorAwardWins'] )
actorDataFrame.to_csv('actors.csv', index = False)
actorDataFrame